In [1]:
import sys
import os
import pandas as pd
import warnings
import json
from devtools import pprint

# Add the src folder to the path
sys.path.append(os.path.abspath('../..'))
warnings.filterwarnings("ignore", category=FutureWarning)

# Import the classes
from src.data_tool.data_providers.polygon import polygon, prepare_news_for_llm, parse_news_by_date
from src.data_tool.data_models import NewsSentiment, NewsSentimentByDate
from src.database_tool.connect_db import ConnectDB
from src.llm.prompt import prompts as p
from src.llm.api_call import call_llm
from langchain_core.prompts import ChatPromptTemplate

In [2]:
pl = polygon()

POLYGON_API_KEY is set
DB_USER and DB_PASSWORD are set


In [3]:
ticker = 'NVDA'
start_date = '2025-04-01'
end_date = '2025-04-07'

In [4]:
nvdia_news, nvdia_news_df = pl.get_news(ticker, start_date, end_date, limit=1000, strict=False)

Skipped 19 duplicate news records for NVDA.
Data successfully inserted into company_news
Successfully inserted 39 new company news records for NVDA.


In [5]:
nvdia_news_df.sort_index(ascending=False)

,article_url,author,description,polygon_id,insights,keywords,published_utc,tickers,title,publisher,ticker
57,https://www.fool.com/investing/2025/04/07/smar...,Ryan Vanzo,The article discusses the rapid growth of the ...,60e29f352a88db666a8bc27fe4ece5856a70cf3b34f64e...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""artificial intelligence"", ""AI ETF"", ""investm...",2025-04-07 09:39:00,"[""NVDA"", ""AMZN"", ""BAC"", ""BACpB"", ""BACpE"", ""BAC...",The Smartest Artificial Intelligence (AI) ETF ...,The Motley Fool,NVDA
56,https://www.investing.com/analysis/editorial-h...,Thomas Monteiro,The article discusses how to navigate the curr...,4588973ba2767bf95d88ad990674ddd6ed05d5b6d4be2b...,"[{""ticker"": ""AAPL"", ""sentiment"": ""positive"", ""...","[""market crash"", ""stock market"", ""investing st...",2025-04-07 09:35:00,"[""AAPL"", ""NVDA"", ""MNST"", ""CSCO""]",Editorial: How to Navigate a Market Crash,Investing.com,NVDA
55,https://www.fool.com/investing/2025/04/06/trum...,The Motley Fool,The article discusses the impact of Trump's ne...,63d96733bd3b5c9c79e15e573382c463ace3114441a151...,"[{""ticker"": ""GOOG"", ""sentiment"": ""neutral"", ""s...","[""import tariffs"", ""tech giants"", ""long-term i...",2025-04-06 22:20:00,"[""GOOG"", ""GOOGL"", ""AAPL"", ""AMZN"", ""META"", ""MSF...","Trump's ""Liberation Day"" Tariffs Pummel the ""M...",The Motley Fool,NVDA
54,https://www.fool.com/investing/2025/04/06/1-ma...,Harsh Chauhan,Taiwan Semiconductor Manufacturing (TSMC) stoc...,735b6ce7291605cd0d0a3cb5f9113c70ff8826d3732525...,"[{""ticker"": ""TSM"", ""sentiment"": ""positive"", ""s...","[""Taiwan Semiconductor Manufacturing"", ""TSMC"",...",2025-04-06 14:45:00,"[""TSM"", ""NVDA"", ""AVGO"", ""MRVL"", ""AMD""]",1 Magnificent Artificial Intelligence (AI) Sto...,The Motley Fool,NVDA
53,https://www.fool.com/investing/2025/04/06/top-...,Geoffrey Seiler,The article discusses how the recent tariff an...,d2b7876a48f77e01a8b31afc505fd1914a8b804eb19e49...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""Nvidia"", ""Amazon"", ""Meta Platforms"", ""tariff...",2025-04-06 14:15:00,"[""NVDA"", ""AMZN"", ""META""]",3 Top Bargain Tech Stocks Ready for the Next B...,The Motley Fool,NVDA
52,https://www.globenewswire.com/news-release/202...,N/A,"Faruqi & Faruqi, LLP is investigating potentia...",ff7760a1230680a0d32cccc9bf0f7165f9dcc447f2e1e8...,"[{""ticker"": ""MPWR"", ""sentiment"": ""negative"", ""...","[""Monolithic Power Systems"", ""Nvidia"", ""class ...",2025-04-06 13:55:00,"[""MPWR"", ""NVDA""]",Faruqi & Faruqi Reminds Monolithic Power Syste...,GlobeNewswire Inc.,NVDA
51,https://www.fool.com/investing/2025/04/06/is-t...,Robert Izquierdo,"Despite recent stock market volatility, the AI...",eabe8c442addabfd56736d3f3aa32b321a318e405c494b...,"[{""ticker"": ""AMD"", ""sentiment"": ""positive"", ""s...","[""artificial intelligence"", ""AI"", ""CoreWeave"",...",2025-04-06 13:13:00,"[""AMD"", ""CRWV"", ""NVDA"", ""AMZN"", ""GOOG"", ""GOOGL...",Is This Artificial Intelligence IPO a Buy?,The Motley Fool,NVDA
50,https://www.fool.com/investing/2025/04/06/3-re...,Geoffrey Seiler,Nvidia's stock has fallen over 25% from its hi...,984756018b1a0c4318cd2a4903b6c26bb62a85a734baac...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""Nvidia"", ""GPU"", ""AI"", ""data center"", ""cloud ...",2025-04-06 10:30:00,"[""NVDA"", ""AMD"", ""META""]",3 Reasons to Buy Nvidia Stock Like There's No ...,The Motley Fool,NVDA
49,https://www.fool.com/investing/2025/04/06/does...,Keith Speights,Billionaire investor David Tepper reduced his ...,d475c00a69e038e359c1af64fa2aaa72fa007368afc2ad...,"[{""ticker"": ""AMZN"", ""sentiment"": ""negative"", ""...","[""David Tepper"", ""Appaloosa Management"", ""Magn...",2025-04-06 08:51:00,"[""AMZN"", ""META"", ""NVDA"", ""MSFT"", ""GOOG"", ""GOOG...",Does Billionaire David Tepper Know Something W...,The Motley Fool,NVDA
48,https://www.fool.com/investing/2025/04/06/nvid...,John Ballard,Nvidia is 

In [6]:
LLM_news = prepare_news_for_llm(nvdia_news.results)

In [7]:
template = p.news_sentiment_template

prompt = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker})

In [8]:
print(prompt)

messages=[SystemMessage(content='You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide a summary of the news and the sentiment of the news.\n                    You should also identify if the news mentioned any upcoming key events that could impact the stock price.\n                ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Based on the following news, create the investment signal:\n                    News Data for NVDA\n                    [\n  {\n    "tickers": [\n      "AAPL",\n      "NVDA",\n      "MSFT"\n    ],\n    "title": "Meet the Hard Asset That\'s Bigger Than Apple, Nvidia, and Microsoft Combined. One Wall Street Strategist Thinks It Could Hit a $40 Trillion Market Cap.",\n    "description": "The article discusses how gold, a hard asset, has outperformed major tech stocks like Apple, Nvidia, and Microsoft, and could potentially reach a $40 trillion market cap according to a Wall 

In [9]:
model_name = "gemini-2.0-flash"
model_provider = "Gemini"
pydantic_model = NewsSentiment
max_retries = 3

In [10]:
sentiment_result = call_llm(prompt, model_name, model_provider, pydantic_model, max_retries=max_retries)

In [11]:
pprint(sentiment_result)

NewsSentiment(
    short_term_sentiment=ShortTermSentiment(
        short_term_sentiment='neutral',
        short_term_confidence=0.6,
        short_term_reasoning="Recent market volatility and tariff concerns have created short-term uncertainty for Nvidia's stock.",
    ),
    long_term_sentiment=LongTermSentiment(
        long_term_sentiment='positive',
        long_term_confidence=0.7,
        long_term_reasoning=(
            'Nvidia is expected to maintain its leadership in the AI chip market due to its strong technological capab'
            'ilities and market position.'
        ),
    ),
    key_events_outlook=[],
)


In [12]:
print(type(sentiment_result))

<class 'src.data_tool.data_models.NewsSentiment'>


In [13]:
template = p.news_sentiment_by_date_template


In [14]:
sentiment_results_by_date = []
for date, news in parse_news_by_date(nvdia_news.results).items():
    llm_news = prepare_news_for_llm(news)
    prompt_news_by_date = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker, "date": date} )
    print(f'\nProcessing news on {date}: {len(news)} news')
    sentiment_result_by_date = call_llm(prompt_news_by_date, model_name, model_provider, NewsSentimentByDate, max_retries=max_retries)
    sentiment_results_by_date.append(sentiment_result_by_date)


7 days of news parsed

Processing news on 2025-04-01: 7 news

Processing news on 2025-04-02: 12 news

Processing news on 2025-04-03: 12 news

Processing news on 2025-04-04: 9 news

Processing news on 2025-04-05: 8 news

Processing news on 2025-04-06: 8 news

Processing news on 2025-04-07: 2 news


In [15]:
len(sentiment_results_by_date)

7

In [16]:
for result in sentiment_results_by_date:
    pprint(result)

NewsSentimentByDate(
    summary=(
        "News for NVDA on 2025-04-01 indicates a mixed sentiment. While NVDA's position in AI is strong, concerns exis"
        't about competition, tariffs, and market sell-offs. Some investors reduced NVDA positions due to valuation co'
        'ncerns.'
    ),
    sentiment='negative',
    reasoning=(
        "The news articles present a mixed sentiment towards NVDA. While some articles highlight NVDA's strong positio"
        'n in the AI market and potential for growth, others express concerns about increasing competition, potential '
        'tariffs, and a general market sell-off affecting semiconductor stocks. Additionally, some billionaire investo'
        'rs have reduced their positions in NVDA, citing high valuations. Overall, the news leans slightly negative du'
        "e to the combination of market headwinds and specific concerns about NVDA's valuation and competitive landsca"
        'pe.'
    ),
    date='2025-04-01',
    stock_pric